In [1]:

import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
%matplotlib inline
sns.set(color_codes=True)


In [2]:
sqlite_file = 'mydb.db'

# Connect to the database and get a cursor object
conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()


In [3]:
cursor.execute("ANALYZE;")
cursor.execute("SELECT tbl, stat FROM sqlite_stat1;")
rows = cursor.fetchall()
db_stats = pd.DataFrame(rows[::-1], columns=['Table','Stat'])
db_stats

,Table,Stat
0,nodes_tags,342727
1,nodes,8694751
2,ways_nodes,10267457
3,ways_tags,6375985
4,ways,1225807


get all tables names and all column names

In [55]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_names =cursor.fetchall()
table_names=[i[0] for i in table_names]
col_names={}
for i in table_names:
    cursor.execute('PRAGMA TABLE_INFO({})'.format(i))
    col_names[i]=[tup[1] for tup in cursor.fetchall()]
pprint.pprint(col_names)


{'nodes': ['id',
           'lat',
           'lon',
           'user',
           'uid',
           'version',
           'changeset',
           'timestamp'],
 'nodes_tags': ['id', 'key', 'value', 'category'],
 'sqlite_stat1': ['tbl', 'idx', 'stat'],
 'ways': ['id', 'user', 'uid', 'version', 'changeset', 'timestamp'],
 'ways_nodes': ['id', 'node_id', 'position'],
 'ways_tags': ['id', 'key', 'value', 'category']}


In [56]:
cursor.execute("SELECT uid, user, sum(count) as count FROM \
               (SELECT uid, user, count(*) as count FROM nodes GROUP BY uid \
               UNION \
               SELECT uid, user, count(*) as count FROM ways GROUP BY uid) \
               GROUP BY uid \
               ORDER BY count desc LIMIT 10;")
rows = cursor.fetchall()
users = pd.DataFrame(rows, columns=['User ID', 'User', 'Count'])
percentages = (users['Count'] / total_nodes_ways * 100).round(decimals=2)
users = pd.concat([users, percentages], axis=1)
users.columns = ['User ID', 'User', 'Contributions', '% of Total']
users


,User ID,User,Contributions,% of Total
0,674454,chicago-buildings,5606367,56.51
1,567034,Umbugbene,1093941,11.03
2,130794,alexrudd (NHD),226078,2.28
3,147510,woodpeck_fixbot,220460,2.22
4,169600,patester24,105172,1.06
5,238419,g246020,104412,1.05
6,187130,mpinnau,103701,1.05
7,5387019,Oak_Park_IL,102949,1.04
8,522978,asdf1234,101109,1.02
9,120146,TIGERcnl,97078,0.98


In [79]:
cursor.execute("SELECT key ,count(key) as count FROM nodes_tags GROUP BY key \
                UNION \
                SELECT key ,count(key) as count FROM ways_tags GROUP BY key \
                ORDER BY count DESC")
rows=cursor.fetchall()
all_keys = pd.DataFrame(rows, columns=['keys', 'Count'])

In [85]:
all_keys[100:150]

,keys,Count
100,lit,3209
101,sidewalk,3175
102,state,3029
103,name_base_2,3025
104,religion,2985
105,cycleway,2766
106,way_id,2734
107,FType,2730
108,hgv,2679
109,attribution,2624


I'm interested in shop and amenity, cuisine,leisure

In [91]:
cursor.execute("SELECT value, sum(count) as count FROM \
               (SELECT value, count(*) as count FROM nodes_tags WHERE key = 'amenity' \
               GROUP BY value \
               UNION \
               SELECT value, count(*) as count FROM ways_tags WHERE key = 'amenity' \
               GROUP BY value) \
               GROUP BY value ORDER BY count desc LIMIT 20;")
rows = cursor.fetchall()
amenities = pd.DataFrame(rows, columns=['Amenity', 'Count'])
amenities

,Amenity,Count
0,parking,13061
1,place_of_worship,4368
2,school,3439
3,restaurant,2176
4,fast_food,1470
5,fuel,1002
6,bank,668
7,cafe,512
8,grave_yard,447
9,shelter,437


In [87]:

cursor.execute("SELECT value, sum(count) as count FROM \
               (SELECT value, count(*) as count FROM nodes_tags WHERE key = 'cuisine' \
               GROUP BY value \
               UNION \
               SELECT value, count(*) as count FROM ways_tags WHERE key = 'cuisine' \
               GROUP BY value) \
               GROUP BY value ORDER BY count desc LIMIT 10;")
rows = cursor.fetchall()
cuisines = pd.DataFrame(rows, columns=['Cuisine', 'Count'])
cuisines.head()

,Cuisine,Count
0,burger,548
1,mexican,245
2,pizza,239
3,sandwich,205
4,american,159


In [89]:
cursor.execute("SELECT value, sum(count) as count FROM \
               (SELECT value, count(*) as count FROM nodes_tags WHERE key = 'leisure' \
               GROUP BY value \
               UNION \
               SELECT value, count(*) as count FROM ways_tags WHERE key = 'leisure' \
               GROUP BY value) \
               GROUP BY value ORDER BY count desc LIMIT 10;")
rows = cursor.fetchall()
Leisure = pd.DataFrame(rows, columns=['Leisure', 'Count'])
Leisure

,Leisure,Count
0,pitch,5926
1,park,4045
2,playground,2960
3,recreation_ground,1287
4,swimming_pool,506
5,golf_course,328
6,sports_centre,277
7,garden,215
8,picnic_table,149
9,nature_reserve,135


In [90]:

cursor.execute("SELECT value, sum(count) as count FROM \
               (SELECT value, count(*) as count FROM nodes_tags WHERE key = 'shop' \
               GROUP BY value \
               UNION \
               SELECT value, count(*) as count FROM ways_tags WHERE key = 'shop' \
               GROUP BY value) \
               GROUP BY value ORDER BY count desc LIMIT 10;")
rows = cursor.fetchall()
Shop = pd.DataFrame(rows, columns=['shop', 'Count'])
Shop

,shop,Count
0,supermarket,578
1,convenience,347
2,clothes,252
3,car_repair,187
4,car,144
5,department_store,136
6,hairdresser,124
7,alcohol,108
8,bakery,100
9,bicycle,96


In [4]:
conn.close()